### Installing necessary libraries

### Getting the dataset for Finetuning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Loading the JSON data and then extracting questions and answers and storing them into a pandas dataframe in separate columns

In [ ]:
import pandas as pd
import json

In [ ]:
with open('/content/drive/MyDrive/train_webmd_squad_v2_consec (1).json', 'r') as json_file:
    data = json.load(json_file)

In [ ]:
def extract_question_context_answer_triples(data):
    question_context_answer_triples = []

    for item in data['data']:
        for paragraph in item['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answers = [answer_info['text'] for answer_info in qa['answers']]
                for answer in answers:
                    question_context_answer_triples.append((question, context, answer))

    return question_context_answer_triples


In [ ]:
question_context_answer_triples = extract_question_context_answer_triples(data)


In [ ]:
df = pd.DataFrame(question_context_answer_triples, columns=['Question', 'Context', 'Answer'])


In [ ]:
with open('/content/drive/MyDrive/test_webmd_squad_v2_full.json', 'r') as json_file:
    data = json.load(json_file)

In [ ]:
question_context_answer_triples = extract_question_context_answer_triples(data)
df2 = pd.DataFrame(question_context_answer_triples, columns=['Question', 'Context', 'Answer'])
df2.shape


(3493, 3)

In [ ]:
df = pd.concat([df, df2], ignore_index=True)
df.shape

(23482, 3)

In [ ]:
with open('/content/drive/MyDrive/val_webmd_squad_v2_full.json', 'r') as json_file:
    data = json.load(json_file)

In [ ]:
question_context_answer_triples = extract_question_context_answer_triples(data)
df3 = pd.DataFrame(question_context_answer_triples, columns=['Question', 'Context', 'Answer'])
df3.shape

(3587, 3)

In [ ]:
df = pd.concat([df, df3], ignore_index=True)
df.shape

(27069, 3)

In [ ]:
df.head()

,Question,Context,Answer
0,What surgical techniques are used to treat gla...,Treatment of open-angle glaucoma -- the most c...,If the glaucoma does not respond to medication...
1,What are the best ways to treat glaucoma?,Treatment of open-angle glaucoma -- the most c...,Both drugs and surgery have high rates of succ...
2,What should you know about treating open-angle...,Treatment of open-angle glaucoma -- the most c...,That is why it's so important to have your eye...
3,Is surgery for glaucoma dangerous?,Treatment of open-angle glaucoma -- the most c...,"Before giving your consent, always ask the sur..."
4,How is acute closed-angle glaucoma treated?,Treatment of open-angle glaucoma -- the most c...,Acute angle-closure glaucoma is different from...


In [ ]:
df.shape

(27069, 3)

#### Saving the file for future reference

In [ ]:
df.to_csv("/content/drive/MyDrive/QA.csv", index = False)

In [ ]:
df.head()

,Question,Context,Answer
0,What surgical techniques are used to treat gla...,Treatment of open-angle glaucoma -- the most c...,If the glaucoma does not respond to medication...
1,What are the best ways to treat glaucoma?,Treatment of open-angle glaucoma -- the most c...,Both drugs and surgery have high rates of succ...
2,What should you know about treating open-angle...,Treatment of open-angle glaucoma -- the most c...,That is why it's so important to have your eye...
3,Is surgery for glaucoma dangerous?,Treatment of open-angle glaucoma -- the most c...,"Before giving your consent, always ask the sur..."
4,How is acute closed-angle glaucoma treated?,Treatment of open-angle glaucoma -- the most c...,Acute angle-closure glaucoma is different from...


#### Changing the format of the Q and A into one column to make it LLM understandable
- f"###Human:\nfind the answer to the {question}\n\n###Assistant:\n{answer}

In [ ]:
df['text'] = df.apply(lambda row: row['Question'] + " " + row['Answer'] + "\n", axis = 1)

In [ ]:
df.head()

,Question,Context,Answer,text
0,What surgical techniques are used to treat gla...,Treatment of open-angle glaucoma -- the most c...,If the glaucoma does not respond to medication...,What surgical techniques are used to treat gla...
1,What are the best ways to treat glaucoma?,Treatment of open-angle glaucoma -- the most c...,Both drugs and surgery have high rates of succ...,What are the best ways to treat glaucoma? Both...
2,What should you know about treating open-angle...,Treatment of open-angle glaucoma -- the most c...,That is why it's so important to have your eye...,What should you know about treating open-angle...
3,Is surgery for glaucoma dangerous?,Treatment of open-angle glaucoma -- the most c...,"Before giving your consent, always ask the sur...",Is surgery for glaucoma dangerous? Before givi...
4,How is acute closed-angle glaucoma treated?,Treatment of open-angle glaucoma -- the most c...,Acute angle-closure glaucoma is different from...,How is acute closed-angle glaucoma treated? Ac...


In [ ]:
from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(df),
})

### Loading the Model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

#### Loading the tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'


In [ ]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","v_proj"]
)

### Loading the Trainer

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 32
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 250
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict['train'],
    # train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    # dataset_text_field="prediction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/27069 [00:00<?, ? examples/s]

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

### Lets Train

In [ ]:
trainer.train()

wandb: Currently logged in as: sarmadafzalj (mlopsassignment). Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
250,1.540000
500,1.476400


TrainOutput(global_step=500, training_loss=1.5081795654296875, metrics={'train_runtime': 3035.3865, 'train_samples_per_second': 21.085, 'train_steps_per_second': 0.165, 'total_flos': 1.5095610649079808e+17, 'train_loss': 1.5081795654296875, 'epoch': 2.36})

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
from peft import LoraConfig, get_peft_model
model = get_peft_model(model, lora_config)

In [ ]:
train_dataset_dict['train']['text'][0]

"What surgical techniques are used to treat glaucoma? If the glaucoma does not respond to medication, or if you cannot tolerate the side effects, your doctor may change medications or recommend one of several surgical techniques: Laser trabeculoplasty creates small laser burns in the area where the fluid drains, improving the outflow rate of aqueous fluid. This relatively brief procedure can often be done in an ophthalmologist's clinic. Trabeculectomy is a surgical procedure that creates a new channel for fluid outflow in cases in which the intraocular pressure is high and the optic nerve damage progresses. Long-term results vary, but generally, the success rate is good. Surgical implants that shunt fluid out of the eye may also be used to decrease pressure in the eye. Remember, all forms of medical or surgical treatment have potential benefits and risks. Before giving your consent, always ask the surgeon to clearly explain any treatment or surgery as well as the proposed benefits, eff

In [ ]:
text = "What types of exercise are best for people with asthma"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=250)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

What types of exercise are best for people with asthma?
 nobody knows for sure.
What is the best exercise for people with asthma?
What is the best exercise for people with asthma? The best exercise for people with asthma is the one that they enjoy and can do without difficulty.
What is the best exercise for people with asthma? The best exercise for people with asthma is the one that they enjoy and can do without difficulty. If you have asthma, you may be concerned about exercise. You may be worried that exercise will trigger an asthma attack.
What is the best exercise for people with asthma? The best exercise for people with asthma is the one that they enjoy and can do without difficulty. If you have asthma, you may be concerned about exercise. You may be worried that exercise will trigger an asthma attack. But exercise can actually help control your asthma.
What is the best exercise for people with asthma? The best exercise for people with asthma is the one that they enjoy and can do 

In [ ]:
text = "Question How is obsessive-compulsive disorder diagnosed?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=250)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Question How is obsessive-compulsive disorder diagnosed?
 nobody knows.
I'm not sure if this is the right place to post this, but I'm going to post it anyway.
I'm a 20 year old male, and I've been having some problems with OCD for the past 2 years. I've been to a psychologist, and he diagnosed me with OCD. I've been on medication for the past 6 months, and I've been doing pretty well. I've been doing pretty well, but I've been having some problems with my medication. I've been having some problems with my medication, and I've been having some problems with my medication. I've been having some problems with my medication, and I've been having some problems with my medication. I've been having some problems with my medication, and I've been having some problems with my medication. I've been having some problems with my medication, and I've been having some problems with my medication. I've been having some problems with my medication, and I've been having some problems with my medication

In [ ]:
text = "When are you more likely to get a blood clot?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=250)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

When are you more likely to get a blood clot?
 nobody knows.
The only thing we know is that it is more likely to happen when you are inactive.
So, if you are inactive, you are more likely to get a blood clot.
If you are inactive, you are more likely to get a blood clot.
If you are inactive, you are more likely to get a blood clot.
If you are inactive, you are more likely to get a blood clot.
If you are inactive, you are more likely to get a blood clot.
If you are inactive, you are more likely to get a blood clot.
If you are inactive, you are more likely to get a blood clot.
If you are inactive, you are more likely to get a blood clot.
If you are inactive, you are more likely to get a blood clot.
If you are inactive, you are more likely to get a blood clot.
If you are inactive, you are more likely to get a blood clot.
If you are inactive, you are more likely to get a blood clot.
If you are inactive, you


In [ ]:
text = "How should you lift objects to prevent back pain?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

How should you lift objects to prevent back pain?
 nobody knows.
The best way to lift objects is to use your legs, not your back.
The best way to lift objects is to use your legs, not your back. The best way to lift objects is to use your legs,


In [ ]:
text = "How can you be smart with antibiotics?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=250)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

How can you be smart with antibiotics?
 nobody knows how many people are affected by antibiotic resistance.
Antibiotic resistance is a global problem.
Antibiotic resistance is a global problem. It is a problem that affects everyone, everywhere.
Antibiotic resistance is a global problem. It is a problem that affects everyone, everywhere. It is a problem that affects everyone, everywhere.
Antibiotic resistance is a global problem. It is a problem that affects everyone, everywhere. It is a problem that affects everyone, everywhere. It is a problem that affects everyone, everywhere. It is a problem that affects everyone, everywhere. It is a problem that affects everyone, everywhere. It is a problem that affects everyone, everywhere. It is a problem that affects everyone, everywhere. It is a problem that affects everyone, everywhere. It is a problem that affects everyone, everywhere. It is a problem that affects everyone, everywhere. It is a problem that affects everyone, everywhere. It is 

In [ ]:
df.head()

,Question,Context,Answer,text
0,What surgical techniques are used to treat gla...,Treatment of open-angle glaucoma -- the most c...,If the glaucoma does not respond to medication...,###Question: \nWhat surgical techniques are us...
1,What are the best ways to treat glaucoma?,Treatment of open-angle glaucoma -- the most c...,Both drugs and surgery have high rates of succ...,###Question: \nWhat are the best ways to treat...
2,What should you know about treating open-angle...,Treatment of open-angle glaucoma -- the most c...,That is why it's so important to have your eye...,###Question: \nWhat should you know about trea...
3,Is surgery for glaucoma dangerous?,Treatment of open-angle glaucoma -- the most c...,"Before giving your consent, always ask the sur...",###Question: \nIs surgery for glaucoma dangero...
4,How is acute closed-angle glaucoma treated?,Treatment of open-angle glaucoma -- the most c...,Acute angle-closure glaucoma is different from...,###Question: \nHow is acute closed-angle glauc...


In [ ]:
search_text = "How is obsessive-compulsive disorder diagnosed?"
result = df[df['text'].str.contains(search_text)]


In [ ]:
result

,Question,Context,Answer,text
23022,How is obsessive-compulsive disorder diagnosed?,var s_context; s_context= s_context || {}; s_c...,You are more likely to get the disorder if you...,###Question: \nHow is obsessive-compulsive dis...
